In [1]:
import sys 
sys.path.append('../src')


In [2]:
from hifigan_module import HiFiGanModuel

hifigan_module = HiFiGanModuel.load_from_checkpoint('../models//hifigan_config1-epoch=969-val_avg_loss=0.2610.ckpt', map_location='cuda:0')


In [3]:
generator = hifigan_module.generator
generator.remove_weight_norm()

Removing weight norm...


In [4]:
import torch
import torchaudio
from torchaudio.transforms import Resample
from audio_processing import mel_spectrogram
def get_item_hifi_gan(audiopath):
    sampling_rate=22050
    ori_sampling_rate=44100
    resampler = Resample(
        orig_freq=ori_sampling_rate,
        new_freq=sampling_rate
    )
    audio, _ = torchaudio.load(audiopath)
    if audio.size(0) !=1:
        audio = torch.mean(audio, dim=0).unsqueeze(0)
    audio = resampler(audio)
    audio = torch.autograd.Variable(audio, requires_grad=False)
    melspec = mel_spectrogram(
        y=audio,
        n_fft=1024,
        num_mels=80,
        sampling_rate=sampling_rate,
        hop_size=256,
        win_size=1024,
        fmin=0.0,
        fmax=11025.0,
    )
    mel_spec = torch.squeeze(melspec, 0)
    return mel_spec.squeeze()


In [5]:
import torch
import torchaudio
from scipy.io.wavfile import write
import os
MAX_WAV_VALUE = 32768.0 
generator.eval()
with torch.no_grad():
    sampling_rate=22050
    mel = get_item_hifi_gan('4_3189.wav')
    mel = mel.to('cuda:0')
    y_g_hat = generator(mel.unsqueeze(0))
    audio = y_g_hat.squeeze()
    audio = audio * MAX_WAV_VALUE
    audio = audio.cpu().numpy().astype('int16')

    output_file = '4_3189_generated.wav'
    write(output_file, sampling_rate, audio)

